In [23]:
# THIS IS OUR BASELINE MODEL.  STRIPS OUT THE BIO TAGS, which is an enhancement.

import operator
import re
from sklearn.metrics import f1_score, accuracy_score


class ScienceNerBaseline():
    
    # load the training set
    def load(self,inFile) :
        txtfile = open(inFile, "r")
        self.tokens = []
        for i in txtfile :
            if i.strip() == "" :
                continue
            wd, tag = i.strip().split()
            
            # the raw text contains the B/I/O tags which we do not want to count in the baseline
            if (tag == "O") :
                tag_pred = ""
            else :
                m = re.match("(.*)-(.*)", tag)
                if m != None :
                    tag_pred = m.group(2)
            self.tokens.append((wd, tag_pred))
        txtfile.close()
        
    
    # this function classifies the 1-gram to be whatever tag the current word is
    def countUnigrams(self):
        self.counts = defaultdict(lambda: defaultdict(lambda: 0.0))
        self.context_totals = dict()
        for word in self.tokens:
            
            # word and tag
            wd, tk = word
            self.counts[wd][tk] += 1

    
    # this function loads the test set and calculates accuracy
    def evaluate(self, testfile) :   
        y_pred = []
        y_true = []
        txtfile = open(testfile, "r")
        testtokens = []
        for i in txtfile :
            
            if i.strip() == "" :
                continue
            wd, tag = i.strip().split()
            testtokens.append((wd, tag))
        txtfile.close()
        
                
        num_correct = num_test = 0
        for word in testtokens:
            tag_true = ""
            wd, tk = word

            m = re.match("(.*)-(.*)", tk)
            if m != None :
                tag_true = m.group(2)
            
            # the prediction is the tag with the most counts for the word
            if wd is not "<UNK>":
                if wd not in self.counts :
                    prediction = [""]
                else :
                    prediction = max(self.counts[wd].iteritems())
                
                # correct prediction
                y_pred += [prediction[0]]
                y_true += [tag_true]
                                
            
        # find the number of labeled predictions - for use in precision calculation
        num_pos_predictions = len([pr for pr in y_pred if pr!=""])
        
        # find number of true labels - for use in recall calculation
        num_true_labels = len([pr for pr in y_true if pr!=""])

        num_correct = numcorrect_all = 0
        for (a,b) in zip(y_pred,y_true) :
            if (a == b) :
                numcorrect_all+=1
                if (a != "") :
                    
                    # get the number of cases correctly labeled
                    num_correct+=1

        precision = num_correct * 1.0 / num_pos_predictions
        recall = num_correct * 1.0 / num_true_labels
        accuracy = numcorrect_all * 1.0 / len(y_pred)
        f1 = 2 * precision * recall / (precision + recall)
        

        return (f1, precision, accuracy)
        

In [24]:
import os
from collections import defaultdict

modelInst = ScienceNerBaseline()

#os.chdir('..')
#os.chdir('w266_project')
#os.chdir('data')
modelInst.load("train.txt")
modelInst.countUnigrams()
f1, precision, acc = modelInst.evaluate("test.txt")
print "Accuracy is :", acc
print "F1 is :", f1
print "Precision is :", precision



Accuracy is : 0.274378585086
F1 is : 0.0641595731165
Precision is : 0.0379827761273
